In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from seminartools.models.four_quarter_average_model import FourQuarterAverageModel
from seminartools.data import read_merged

# 1. Read data 

In [3]:
# expanding window crossval
from seminartools.time_series_split import ExpandingWindowSplit

tss = ExpandingWindowSplit(start_date = "2000-01-01")
tss

In [4]:
df = read_merged(coreInf=True).reset_index()
df

,country,date,inflation,gdp_growth,interest_rate,unemployment_rate,commodity_CRUDE_PETRO,commodity_iNATGAS,commodity_iAGRICULTURE,commodity_iMETMIN,commodity_iPRECIOUSMET
0,United States,1977-01-01,0.012346,1.186233,4.700000,8.2,0.00000,0.052425,0.144757,0.041940,0.112740
1,Japan,1977-01-01,0.016420,2.187257,6.333333,2.3,0.00000,0.052425,0.144757,0.041940,0.112740
2,Canada,1977-01-01,0.018797,1.533830,8.166667,8.8,0.00000,0.052425,0.144757,0.041940,0.112740
3,Japan,1977-04-01,0.030695,0.703788,5.000000,2.0,-0.00400,0.043586,-0.066688,-0.045412,-0.060029
4,Canada,1977-04-01,0.022140,0.510302,7.666667,8.0,-0.00400,0.043586,-0.066688,-0.045412,-0.060029
...,...,...,...,...,...,...,...,...,...,...,...
1317,Germany,2023-01-01,0.012626,0.104301,3.000000,3.0,-0.02041,-0.585458,0.002037,0.007418,0.043183
1318,Switzerland,2023-01-01,0.006717,0.309543,1.166667,4.4,-0.02041,-0.585458,0.002037,0.007418,0.043183
1319,Canada,2023-01-01,0.007967,0.610026,4.500000,5.4,-0.02041,-0.585458,0.002037,0.007418,0.043183
1320,Sweden,2023-01-01,0.025074,0.656897,2.833333,7.8,-0.02041,-0.585458,0.002037,0.007418,0.043183


# 2. get summary stats for all models

In [6]:
from seminartools.models.utils import get_stats
from seminartools.models.four_quarter_average_model import FourQuarterAverageModel
from seminartools.models.pca_var_model import PCAVARModel
from seminartools.models.armax_model import ARMAXModel
from seminartools.models.uc_sv_ss_model import UCSVSSModel
from seminartools.models.random_effects_model import RandomEffectsModel
from seminartools.models.distance_model import DistanceModel
from seminartools.models.mucsvss_model import MUCSVSSModel

muc_model = MUCSVSSModel(num_particles=100008, stochastic_seasonality=True)
muc_model.load_from_disk("../../models/mucsvss_model_100008_deterministic_all.pkl")
muc_model

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pycountry/db.py:51: UserWarning: Country's official_name not found. Country name provided instead.
  warnings.warn(warning_message, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pycountry/db.py:51: UserWarning: Country's common_name not found. Country name provided instead.
  warnings.warn(warning_message, UserWarning)


In [7]:
exog_columns = [
    "gdp_growth",
    "interest_rate",
    "unemployment_rate",
    "commodity_CRUDE_PETRO",
    "commodity_iNATGAS",
    "commodity_iAGRICULTURE",
    "commodity_iMETMIN",
    "commodity_iPRECIOUSMET",
]

stats = get_stats(
    [
        ("4qa", lambda: FourQuarterAverageModel()),
        #("pca_var", lambda: PCAVARModel()),
        ("ar", lambda: ARMAXModel(max_p=2, max_q=0)),
        (
            "arx",
            lambda: ARMAXModel(max_p=2, max_q=0, exogenous_columns=exog_columns),
        ),
        ("arma", lambda: ARMAXModel(max_p=2, max_q=2)),
        (
            "armax",
            lambda: ARMAXModel(max_p=2, max_q=2, exogenous_columns=exog_columns),
        ),
        (
            "uc_sv",
            lambda: UCSVSSModel(num_particles=10000, stochastic_seasonality=False),
        ),
        (
            "uc_sv_ss",
            lambda: UCSVSSModel(num_particles=10000, stochastic_seasonality=True),
        ),
        ("random_effects", lambda: RandomEffectsModel()),
        (
            "random_effects_x",
            lambda: RandomEffectsModel(exogenous_columns=exog_columns),
        ),
        ("distance", lambda: DistanceModel()),
        ("distance_x", lambda: DistanceModel(exogenous_columns=exog_columns)),
        ("mucsvss", lambda: muc_model),
    ],
    df,
    tss,
    num_cores_parallel_models=1,  # only 1 model at once because models use joblib internally and nested joblib gives issues
)
stats

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/base/model.py:1914: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/base/model.py:1914: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior

,mse,mae,r2,mz_intercept,mz_slope,mz_r2,mz_wald_test_statistic,p-value
arma,0.000027,0.003805,0.215571,0.001268,0.653971,0.299434,50.995610,1.2060416924271445e-21
4qa,0.000028,0.003922,0.173718,0.000835,0.831466,0.182588,5.458379,0.00438758963658192
ar,0.000029,0.004031,0.146727,0.001045,0.658756,0.205244,31.365997,7.179245566959854e-14
arx,0.000063,0.005925,-0.833421,0.003116,0.193604,0.050391,396.482816,1.925122742658902e-122
armax,0.001795,0.019629,-51.081160,0.003728,0.003852,0.000756,21777.364759,0.0


In [11]:
# format r2 column to 2 
format_stats = stats.copy()
format_stats['p-value'] =format_stats['p-value'].astype(float)
round_cols = ["r2", "mz_intercept", "mz_slope", "mz_r2", "mz_wald_test_statistic", "p-value"]
for col in round_cols:
    format_stats[col] = format_stats[col].apply(lambda x: round(x, 3))

format_stats['mz_wald_test_statistic'] = format_stats['mz_wald_test_statistic'].astype(str) + '(' + format_stats['p-value'].astype(str) + ')'
format_stats.drop(columns = ['p-value'], inplace = True)
format_stats

,mse,mae,r2,mz_intercept,mz_slope,mz_r2,mz_wald_test_statistic
arma,0.000027,0.003805,0.216,0.001,0.654,0.299,50.996(0.0)
4qa,0.000028,0.003922,0.174,0.001,0.831,0.183,5.458(0.004)
ar,0.000029,0.004031,0.147,0.001,0.659,0.205,31.366(0.0)
arx,0.000063,0.005925,-0.833,0.003,0.194,0.050,396.483(0.0)
armax,0.001795,0.019629,-51.081,0.004,0.004,0.001,21777.365(0.0)


In [12]:
# generate latex code for the stats table
print(
    # replace _ by \_ in columns and index
    format_stats.pipe(lambda df: df.rename(columns=lambda x: x.replace("_", "\\_")))
    .pipe(lambda df: df.rename(index=lambda x: x.replace("_", "\\_")))
    .to_latex(
        index=True,
        caption="MSPE, MAE, $R^2$, and Mincer-Zarnowitz statistics, out-of-sample, 2000-2023",
        label="tab:model_eval",
        float_format="%.4f",
    )
)

\begin{table}
\caption{MSPE, MAE, $R^2$, and Mincer-Zarnowitz statistics, out-of-sample, 2000-2023}
\label{tab:model_eval}
\begin{tabular}{lrrrrrrl}
\toprule
 & mse & mae & r2 & mz\_intercept & mz\_slope & mz\_r2 & mz\_wald\_test\_statistic \\
\midrule
arma & 0.0000 & 0.0038 & 0.2160 & 0.0010 & 0.6540 & 0.2990 & 50.996(0.0) \\
4qa & 0.0000 & 0.0039 & 0.1740 & 0.0010 & 0.8310 & 0.1830 & 5.458(0.004) \\
ar & 0.0000 & 0.0040 & 0.1470 & 0.0010 & 0.6590 & 0.2050 & 31.366(0.0) \\
arx & 0.0001 & 0.0059 & -0.8330 & 0.0030 & 0.1940 & 0.0500 & 396.483(0.0) \\
armax & 0.0018 & 0.0196 & -51.0810 & 0.0040 & 0.0040 & 0.0010 & 21777.365(0.0) \\
\bottomrule
\end{tabular}
\end{table}

